In [1]:
!pip install PyDrive

     |████████████████████████████████| 993kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':'1ht3cKlNfTNsJ7kE9nKSsQ1sDMh8jeTM5'})
download.GetContentFile('Training_SSD.zip')

In [0]:
download = drive.CreateFile({'id':'10Dcc0prC0_t7uX2TjH6GB2d3KkQr0ppw'})
download.GetContentFile('ssd300_mAP_77.43_v2.pth')

In [0]:
# need to upload file from computer named ssd.py  which resides on this address 
# D:\videos\Udemy - computer-vision-a-z\06 Module 2 - Object Detection with SSD\Training_SSD

In [0]:
!unzip Training_SSD.zip

In [13]:
cd Training_SSD/


/content/Training_SSD


In [0]:
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform
from data import VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

In [0]:
# Defining a function that will do the detections
def detect(frame, net, transform):
    height, width = frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    y = net(x)
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    # detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            pt = (detections[0, i, j, 1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            j += 1
    return frame

In [19]:
# Creating the SSD neural network
net = build_ssd('test')
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))

/content/Training_SSD/ssd.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/content/Training_SSD/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
# Creating the transformation
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [21]:
from google.colab import files
uploaded = files.upload() 
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

Saving 036 epic-horses.mp4 to 036 epic-horses.mp4
saved file 036 epic-horses.mp4


In [26]:
# Doing some Object Detection on a video
reader = imageio.get_reader('036 epic-horses.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output.mp4', fps = fps)
for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame)
    print(i)
writer.close()

/content/Training_SSD/ssd.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.softmax(conf.view(-1, self.num_classes)),  # conf preds


ValueError: ignored